In [4]:
# Complete, robust Gradio Week-1 app for Colab (copy-paste into ONE cell)
!pip install -q gradio pandas

import sqlite3, pandas as pd, datetime, threading, traceback, sys, os
import gradio as gr

# ---------- Config & DB setup ----------
DB_PATH = "health_data.db"
# allow access from multiple threads (Gradio worker threads)
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
conn.row_factory = sqlite3.Row
cur = conn.cursor()
db_lock = threading.Lock()  # prevent concurrent writes

def init_db():
    with db_lock:
        cur.execute('''
        CREATE TABLE IF NOT EXISTS medications (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            time TEXT NOT NULL,
            taken INTEGER DEFAULT 0
        )
        ''')
        cur.execute('''
        CREATE TABLE IF NOT EXISTS health_metrics (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            date TEXT NOT NULL,
            steps INTEGER DEFAULT 0,
            calories INTEGER DEFAULT 0
        )
        ''')
        conn.commit()

init_db()

# ---------- Helper utilities ----------
def log_exception(fn_name="unknown"):
    tb = traceback.format_exc()
    with open("last_error.log", "a") as fh:
        fh.write(f"\n[{datetime.datetime.now().isoformat()}] Exception in {fn_name}:\n")
        fh.write(tb + "\n")
    print(f"Exception logged to last_error.log from {fn_name}", file=sys.stderr)
    # keep the actual traceback in the file; don't spam the UI with huge traces

def df_or_message(df):
    if df is None or (isinstance(df, pd.DataFrame) and df.empty):
        return "No records found."
    return df

# ---------- CRUD functions (safe & validated) ----------
def add_med(name, time_str):
    """Add a medication; returns status string."""
    try:
        if not name or str(name).strip() == "":
            return "Error: medicine name is required."
        if not time_str or str(time_str).strip() == "":
            return "Error: time is required (HH:MM)."

        t_raw = str(time_str).strip()
        parts = t_raw.split(":")
        if len(parts) < 2:
            return "Error: time must be in HH:MM format (e.g. 08:30)."

        try:
            hh = int(parts[0])
            mm = int(parts[1])
        except ValueError:
            return "Error: time contains non-numeric values. Use HH:MM (e.g. 08:30)."

        if not (0 <= hh <= 23 and 0 <= mm <= 59):
            return "Error: hour must be 0-23 and minute 0-59."

        tnorm = f"{hh:02d}:{mm:02d}"
        name_clean = str(name).strip()

        with db_lock:
            cur.execute("INSERT INTO medications (name, time) VALUES (?,?)", (name_clean, tnorm))
            conn.commit()

        return f"Added '{name_clean}' at {tnorm}."

    except Exception:
        log_exception("add_med")
        return "Error: an unexpected error occurred while adding medication. See last_error.log."

def view_meds():
    """Return pandas DataFrame of meds or a friendly message string (Gradio can accept DataFrame)."""
    try:
        df = pd.read_sql("SELECT * FROM medications ORDER BY time", conn)
        return df_or_message(df)
    except Exception:
        log_exception("view_meds")
        return "Error: could not read medications. See last_error.log."

def mark_taken(med_id):
    """Mark a medication taken by id; returns status string."""
    try:
        if med_id is None:
            return "Error: please enter a medication id."
        med_id = int(med_id)
        with db_lock:
            cur.execute("UPDATE medications SET taken=1 WHERE id=?", (med_id,))
            if cur.rowcount == 0:
                return f"Warning: no medication found with id {med_id}."
            conn.commit()
        return f"Marked medication id {med_id} as taken."
    except ValueError:
        return "Error: medication id must be a number."
    except Exception:
        log_exception("mark_taken")
        return "Error: an unexpected error occurred while marking taken. See last_error.log."

def clear_taken():
    try:
        with db_lock:
            cur.execute("UPDATE medications SET taken=0")
            conn.commit()
        return "Reset 'taken' status for all medications."
    except Exception:
        log_exception("clear_taken")
        return "Error: could not reset flags. See last_error.log."

def add_metrics(date_str, steps, calories):
    try:
        if not date_str or str(date_str).strip() == "":
            return "Error: date is required (YYYY-MM-DD)."
        # normalize date
        try:
            datetime.datetime.strptime(str(date_str), "%Y-%m-%d")
        except Exception:
            return "Error: date must be YYYY-MM-DD (e.g. 2025-10-20)."

        steps_v = int(steps) if steps not in (None, "") else 0
        calories_v = int(calories) if calories not in (None, "") else 0

        with db_lock:
            cur.execute("INSERT INTO health_metrics (date, steps, calories) VALUES (?,?,?)",
                        (str(date_str), steps_v, calories_v))
            conn.commit()
        return f"Saved metrics for {date_str}."
    except ValueError:
        return "Error: steps and calories must be whole numbers."
    except Exception:
        log_exception("add_metrics")
        return "Error: an unexpected error occurred while saving metrics. See last_error.log."

def view_metrics():
    try:
        df = pd.read_sql("SELECT * FROM health_metrics ORDER BY date DESC", conn)
        return df_or_message(df)
    except Exception:
        log_exception("view_metrics")
        return "Error: could not read health metrics. See last_error.log."

def check_reminders_now():
    try:
        now = datetime.datetime.now()
        df = pd.read_sql("SELECT * FROM medications WHERE taken=0", conn)
        if df.empty:
            return "No pending medication reminders."
        soon = []
        for _, row in df.iterrows():
            try:
                med_time = datetime.datetime.strptime(row['time'], "%H:%M")
                med_dt = now.replace(hour=med_time.hour, minute=med_time.minute, second=0, microsecond=0)
                diff_min = int((med_dt - now).total_seconds() / 60)
                if abs(diff_min) <= 15:
                    soon.append(f"{row['name']} at {row['time']} (id={row['id']}) — {diff_min} min {'late' if diff_min<0 else 'away'}")
            except Exception:
                continue
        if not soon:
            return "No reminders within ±15 minutes from now."
        return "\n".join(soon)
    except Exception:
        log_exception("check_reminders_now")
        return "Error: could not check reminders. See last_error.log."

# ---------- Build Gradio UI ----------
with gr.Blocks() as demo:
    gr.Markdown("# 🩺 Healthcare Monitoring AI Agent — Week 1 (Colab + Gradio)")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Add Medication")
            m_name = gr.Textbox(label="Medicine name", placeholder="e.g. Paracetamol")
            m_time = gr.Textbox(label="Time (HH:MM)", placeholder="08:30")
            add_btn = gr.Button("Add Reminder")
            add_status = gr.Textbox(label="Status", interactive=False)
            add_btn.click(add_med, inputs=[m_name, m_time], outputs=add_status)

            gr.Markdown("### Medication Actions")
            view_btn = gr.Button("View Schedule")
            meds_table = gr.Dataframe(headers=["id","name","time","taken"])
            view_btn.click(lambda: pd.read_sql("SELECT * FROM medications ORDER BY time", conn), outputs=meds_table)

            med_id = gr.Number(label="Medication id to mark taken")
            mark_btn = gr.Button("Mark Taken")
            mark_out = gr.Textbox(label="Mark status", interactive=False)
            mark_btn.click(mark_taken, inputs=med_id, outputs=mark_out)

            reset_btn = gr.Button("Reset 'taken' flags")
            reset_out = gr.Textbox()
            reset_btn.click(clear_taken, outputs=reset_out)

        with gr.Column(scale=1):
            gr.Markdown("### Health Metrics")
            date_in = gr.Textbox(label="Date (YYYY-MM-DD)", value=datetime.date.today().isoformat())
            steps_in = gr.Number(label="Steps", value=0)
            cal_in = gr.Number(label="Calories", value=0)
            save_metrics = gr.Button("Save Metrics")
            metrics_status = gr.Textbox()
            save_metrics.click(add_metrics, inputs=[date_in, steps_in, cal_in], outputs=metrics_status)

            view_metrics_btn = gr.Button("View Metrics")
            metrics_table = gr.Dataframe(headers=["id","date","steps","calories"])
            view_metrics_btn.click(lambda: pd.read_sql("SELECT * FROM health_metrics ORDER BY date DESC", conn), outputs=metrics_table)

            gr.Markdown("### Reminders")
            check_btn = gr.Button("Check Reminders (±15 min)")
            reminders_out = gr.Textbox(label="Upcoming reminders", interactive=False)
            check_btn.click(check_reminders_now, outputs=reminders_out)

    gr.Markdown("_Notes: DB `health_data.db` is stored in the Colab VM. If the runtime resets you may lose data — download the DB file if you want to keep it._")

# ---------- Launch (share=True shows a public link from Colab) ----------
demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2aa3c49305546ad836.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [5]:
# ✅ Fixed version: Generate app.py and supporting files correctly
app_py = """\
import sqlite3, pandas as pd, datetime, os
import gradio as gr

DB_PATH = "health_data.db"
conn = sqlite3.connect(DB_PATH, check_same_thread=False)
cur = conn.cursor()

# Create tables
cur.execute('''
CREATE TABLE IF NOT EXISTS medications (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    time TEXT NOT NULL,
    taken INTEGER DEFAULT 0
)
''')
cur.execute('''
CREATE TABLE IF NOT EXISTS health_metrics (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    date TEXT NOT NULL,
    steps INTEGER DEFAULT 0,
    calories INTEGER DEFAULT 0
)
''')
conn.commit()

def add_med(name, time_str):
    if not name or not time_str:
        return "Please provide medicine name and time (HH:MM)."
    try:
        hh, mm = map(int, str(time_str).split(":")[:2])
        tnorm = f"{hh:02d}:{mm:02d}"
    except Exception:
        return "Time must be in HH:MM format (e.g. 08:30)."
    cur.execute("INSERT INTO medications (name, time) VALUES (?,?)", (name.strip(), tnorm))
    conn.commit()
    return f"Added '{name.strip()}' at {tnorm}."

def view_meds():
    return pd.read_sql("SELECT * FROM medications ORDER BY time", conn)

def mark_taken(med_id):
    try:
        cur.execute("UPDATE medications SET taken=1 WHERE id=?", (int(med_id),))
        conn.commit()
        return f"Marked medication id {med_id} as taken."
    except Exception as e:
        return f"Error: {e}"

def clear_taken():
    cur.execute("UPDATE medications SET taken=0")
    conn.commit()
    return "Reset 'taken' status for all medications."

def add_metrics(date_str, steps, calories):
    try:
        datetime.datetime.strptime(date_str, "%Y-%m-%d")
    except Exception:
        return "Date must be YYYY-MM-DD."
    cur.execute("INSERT INTO health_metrics (date, steps, calories) VALUES (?,?,?)",
                (date_str, int(steps or 0), int(calories or 0)))
    conn.commit()
    return f"Saved metrics for {date_str}."

def view_metrics():
    return pd.read_sql("SELECT * FROM health_metrics ORDER BY date DESC", conn)

def check_reminders_now():
    now = datetime.datetime.now()
    df = pd.read_sql("SELECT * FROM medications WHERE taken=0", conn)
    if df.empty:
        return "No pending medication reminders."
    soon = []
    for _, row in df.iterrows():
        try:
            med_time = datetime.datetime.strptime(row['time'], "%H:%M")
            med_dt = now.replace(hour=med_time.hour, minute=med_time.minute, second=0, microsecond=0)
            diff = int((med_dt - now).total_seconds() / 60)
            if abs(diff) <= 15:
                soon.append(f"{row['name']} at {row['time']} (id={row['id']}) — {diff} min {'late' if diff<0 else 'away'}")
        except Exception:
            continue
    if not soon:
        return "No reminders within ±15 minutes from now."
    return "\\n".join(soon)

with gr.Blocks() as demo:
    gr.Markdown("## 🩺 Healthcare Monitoring AI Agent — Week 1 (Colab + Gradio)")
    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### Add Medication")
            m_name = gr.Textbox(label="Medicine name", placeholder="e.g. Paracetamol")
            m_time = gr.Textbox(label="Time (HH:MM)", placeholder="08:30")
            add_btn = gr.Button("Add Reminder")
            add_status = gr.Textbox(label="Status", interactive=False)
            add_btn.click(add_med, inputs=[m_name, m_time], outputs=add_status)

            gr.Markdown("### Medication Actions")
            view_btn = gr.Button("View Schedule")
            meds_table = gr.Dataframe(headers=["id","name","time","taken"])
            view_btn.click(lambda: pd.read_sql("SELECT * FROM medications ORDER BY time", conn), outputs=meds_table)

            med_id = gr.Number(label="Medication id to mark taken")
            mark_btn = gr.Button("Mark Taken")
            mark_out = gr.Textbox(label="Mark status", interactive=False)
            mark_btn.click(mark_taken, inputs=med_id, outputs=mark_out)

            reset_btn = gr.Button("Reset 'taken' flags")
            reset_out = gr.Textbox()
            reset_btn.click(clear_taken, outputs=reset_out)

        with gr.Column(scale=1):
            gr.Markdown("### Health Metrics")
            date_in = gr.Textbox(label="Date (YYYY-MM-DD)", value=datetime.date.today().isoformat())
            steps_in = gr.Number(label="Steps", value=0)
            cal_in = gr.Number(label="Calories", value=0)
            save_metrics = gr.Button("Save Metrics")
            metrics_status = gr.Textbox()
            save_metrics.click(add_metrics, inputs=[date_in, steps_in, cal_in], outputs=metrics_status)

            view_metrics_btn = gr.Button("View Metrics")
            metrics_table = gr.Dataframe()
            view_metrics_btn.click(lambda: pd.read_sql("SELECT * FROM health_metrics ORDER BY date DESC", conn), outputs=metrics_table)

            gr.Markdown("### Reminders")
            check_btn = gr.Button("Check Reminders (±15 min)")
            reminders_out = gr.Textbox(label="Upcoming reminders", interactive=False)
            check_btn.click(check_reminders_now, outputs=reminders_out)

    gr.Markdown("_DB file `health_data.db` is saved in the Colab VM. Download it if you want to preserve data._")

demo.launch(share=False)
"""

# Write to file
with open("app.py", "w") as f:
    f.write(app_py)

print("✅ app.py created successfully!")


✅ app.py created successfully!


In [6]:
!zip -r week1_package.zip app.py health_data.db


updating: app.py (deflated 65%)
updating: health_data.db (deflated 98%)
